In [3]:
import numpy as np

import openml
import pandas as pd

In [19]:
from tqdm import tqdm

from datasets import load_openml_list, test_dids_classification, valid_large_classification, open_cc_dids, open_cc_valid_dids


In [ ]:
%load_ext autoreload

%autoreload 2

### Prepare test datasets

In [5]:
renamer = {'name': 'Name', 'NumberOfFeatures': '# Features', 'NumberOfSymbolicFeatures': '# Categorical Features', 'NumberOfInstances': '# Instances', 'NumberOfMissingValues': '# NaNs', 'NumberOfClasses': '# Classes', 'MinorityClassSize': 'Minority Class Size'}


In [6]:
openml.study.list_suites()

OrderedDict([(99,
              {'id': 99,
               'alias': 'OpenML-CC18',
               'main_entity_type': 'task',
               'name': 'OpenML-CC18 Curated Classification benchmark',
               'status': 'active',
               'creation_date': '2019-02-21 18:47:13',
               'creator': 1}),
             (225,
              {'id': 225,
               'alias': 'OpenML-friendly',
               'main_entity_type': 'task',
               'name': 'OpenML100-friendly',
               'status': 'active',
               'creation_date': '2019-09-16 19:41:46',
               'creator': 1})])

In [7]:
suite = openml.study.get_suite(suite_id=99)
tasks = openml.tasks.list_tasks(output_format="dataframe")

In [8]:
# Using ``@`` in `pd.DataFrame.query <
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html>`_
# accesses variables outside of the current dataframe.
tasks = tasks.query("tid in @suite.tasks")

In [9]:
tids = list(tasks[np.logical_and(np.logical_and((tasks.NumberOfInstances <= 2000), (tasks.NumberOfFeatures <= 100))
                                 , (tasks.NumberOfClasses <= 10))].tid)

In [10]:
len(tids)

30

In [11]:
tids = list(tasks[tasks.NumberOfInstances <= 2000].tid)

In [12]:
open_cc_dids = [openml.tasks.get_task(task_id).get_dataset().id for task_id in tids]

In [13]:
open_ml_datasets, open_ml_datasets_df = load_openml_list(test_dids_classification, multiclass=True, shuffled=True, filter_for_nan=False, max_samples = 100000, num_feats=100, return_capped=True)


Number of datasets: 36
Loading kr-vs-kp 3 ..
Loading mfeat-factors 12 ..
Loading credit-g 31 ..
Loading vehicle 54 ..
Loading wine 973 ..
Loading kc1 1067 ..
Loading KDDCup09_appetency 1111 ..
Loading airlines 1169 ..
Loading bank-marketing 1461 ..
Loading blood-transfusion-service-center 1464 ..
Loading cnae-9 1468 ..
Loading nomao 1486 ..
Loading phoneme 1489 ..
Loading adult 1590 ..
Loading covertype 1596 ..
Loading numerai28.6 23517 ..
Loading connect-4 40668 ..
Loading car 40975 ..
Loading Australian 40981 ..
Loading segment 40984 ..
Loading jungle_chess_2pcs_raw_endgame_complete 41027 ..
Loading APSFailure 41138 ..
Loading christine 41142 ..
Loading jasmine 41143 ..
Loading sylvine 41146 ..
Loading albert 41147 ..
Loading MiniBooNE 41150 ..
Loading guillermo 41159 ..
Loading riccardo 41161 ..
Loading dilbert 41163 ..
Loading fabert 41164 ..
Loading robert 41165 ..
Loading volkert 41166 ..
Loading dionis 41167 ..
Loading jannis 41168 ..
Loading helena 41169 ..


In [14]:
open_ml_datasets_df = open_ml_datasets_df[open_ml_datasets_df.NumberOfInstances > 10000]

In [15]:
print_table = open_ml_datasets_df
print_table = print_table[['name', 'NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].copy()
print_table['id'] = print_table.index
print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']] = print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].astype(int)
print_table = print_table.rename(columns=renamer)
print(print_table.to_latex(index=False))

\begin{tabular}{lrrrrrrr}
\toprule
                                  Name &  \# Features &  \# Categorical Features &  \# Instances &  \# Classes &  \# NaNs &  Minority Class Size &    id \\
\midrule
                    KDDCup09\_appetency &         231 &                      39 &        50000 &          2 & 8024152 &                  890 &  1111 \\
                              airlines &           8 &                       5 &       539383 &          2 &       0 &               240264 &  1169 \\
                        bank-marketing &          17 &                      10 &        45211 &          2 &       0 &                 5289 &  1461 \\
                                 nomao &         119 &                      30 &        34465 &          2 &       0 &                 9844 &  1486 \\
                                 adult &          15 &                       9 &        48842 &          2 &    6465 &                11687 &  1590 \\
                             covertype &    

### Prepare Validation datasets

In [20]:
open_cc_datasets, open_cc_datasets_df = load_openml_list(open_cc_dids, multiclass=True, shuffled=True, filter_for_nan=False, max_samples = 2000, num_feats=100, return_capped=True)

def extend_datasets(datasets, filtering = False):
    extended_datasets = {}
    i = 0
    for d in tqdm(datasets):
        if ((not 'NumberOfFeatures' in datasets[d])
                or (not 'NumberOfClasses' in datasets[d])
                or (not 'NumberOfInstances' in datasets[d])
                # or datasets[d]['NumberOfFeatures'] >= num_feats
                or datasets[d]['NumberOfClasses'] <= 0):
            print(datasets[d])
            continue
        ds = openml.datasets.get_dataset(d, download_data=False)
        if filtering and (datasets[d]['NumberOfInstances'] < 150
                          or datasets[d]['NumberOfInstances'] > 2000
                         or datasets[d]['NumberOfFeatures'] > 100
                         or datasets[d]['NumberOfClasses'] > 10):
            continue
        extended_datasets[d] = datasets[d]
        extended_datasets[d].update(ds.qualities)
    
    return extended_datasets

# All datasets
openml_list = openml.datasets.list_datasets()
openml_list = pd.DataFrame.from_dict(openml_list, orient="index")

# Select only classification
openml_list = openml_list[~openml_list['MajorityClassSize'].isna()]

# Remove duplicated datasets
duplicated = openml_list.duplicated(subset=['MajorityClassSize', 'MaxNominalAttDistinctValues', 'MinorityClassSize',
       'NumberOfClasses', 'NumberOfFeatures', 'NumberOfInstances',
       'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
       'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures'], keep='first')
openml_list = openml_list[~duplicated]

duplicated = openml_list.duplicated(subset=['name'], keep='first')
openml_list = openml_list[~duplicated]

# Filter out datasets that don't have meta information or Don't fulfill other criteria
openml_list = openml_list.to_dict(orient='index')
openml_list = pd.DataFrame.from_dict(extend_datasets(openml_list, filtering=True), orient="index")

# Filter out datasets in Open CC
openml_list = openml_list[~openml_list.name.apply(lambda x: x in test_datasets_multiclass_df.name.values)]
openml_list['CFI'] = openml_list.apply(lambda x: str(x.NumberOfClasses) + '_' + str(x.NumberOfFeatures) + '_' + str(x.NumberOfInstances), axis = 1)
test_datasets_multiclass_df['CFI'] = test_datasets_multiclass_df.apply(lambda x: str(x.NumberOfClasses) + '_' + str(x.NumberOfFeatures) + '_' + str(x.NumberOfInstances), axis = 1)
openml_list = openml_list[~openml_list.CFI.apply(lambda x: x in test_datasets_multiclass_df.CFI.values)]

# Remove time series and artificial data
openml_list = openml_list[~openml_list.name.apply(lambda x: 'autoUniv' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'fri_' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'FOREX' in x)]

# Remove datasets that overlapped with Open CC closely by name
openml_list = openml_list[~openml_list.name.apply(lambda x: 'ilpd' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'car' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'pc1' in x)]

# Remove datasets that didn't load
openml_list = openml_list[~openml_list.did.apply(lambda x: x in {1065, 40589, 41496, 770, 43097, 43148, 43255, 43595, 43786, 41701})]

# Remove class skew
openml_list = openml_list[(openml_list.MinorityClassSize / openml_list.MajorityClassSize) > 0.05]
openml_list = openml_list[openml_list.AutoCorrelation != 1]

# Remove too easy
openml_list = openml_list[openml_list.CfsSubsetEval_DecisionStumpAUC != 1]

Number of datasets: 30
Loading balance-scale 11 ..
Loading mfeat-fourier 14 ..
Loading breast-w 15 ..
Loading mfeat-karhunen 16 ..
Loading mfeat-morphological 18 ..
Loading mfeat-zernike 22 ..
Loading cmc 23 ..
Loading credit-approval 29 ..
Loading credit-g 31 ..
Loading diabetes 37 ..
Loading tic-tac-toe 50 ..
Loading vehicle 54 ..
Loading eucalyptus 188 ..
Loading analcatdata_authorship 458 ..
Loading analcatdata_dmft 469 ..
Loading pc4 1049 ..
Loading pc3 1050 ..
Loading kc2 1063 ..
Loading pc1 1068 ..
Loading banknote-authentication 1462 ..
Loading blood-transfusion-service-center 1464 ..
Loading ilpd 1480 ..
Loading qsar-biodeg 1494 ..
Loading wdbc 1510 ..
Loading cylinder-bands 6332 ..
Loading dresses-sales 23381 ..
Loading MiceProtein 40966 ..
Loading car 40975 ..
Loading steel-plates-fault 40982 ..
Loading climate-model-simulation-crashes 40994 ..


KeyError: 'MajorityClassSize'

In [17]:
print_table = openml_list
print_table = print_table[['name', 'NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].copy()
print_table['id'] = print_table.index
print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']] = print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].astype(int)
print_table = print_table.rename(columns=renamer)
print(print_table.to_latex(index=False))

\begin{tabular}{lrrrrrrr}
\toprule
                                              Name &  \# Features &  \# Categorical Features &  \# Instances &  \# Classes &  \# NaNs &  Minority Class Size &    id \\
\midrule
                                            anneal &          39 &                      33 &          898 &          5 &   22175 &                    8 &     2 \\
                                             autos &          26 &                      11 &          205 &          6 &      59 &                    3 &     9 \\
                                     balance-scale &           5 &                       1 &          625 &          3 &       0 &                   49 &    11 \\
                                     breast-cancer &          10 &                      10 &          286 &          2 &       9 &                   85 &    13 \\
                                     mfeat-fourier &          77 &                       1 &         2000 &         10 &       0 &      